In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException #except문 처리
import requests
import re
import time
import pandas as pd
from bs4 import BeautifulSoup

# 함수 + 예외처리 + 실패 따로 처리 + 구석구석 으로 실행해보기

In [22]:
import pandas as pd
import re

# CSV 파일 읽기
Tour_Place = pd.read_csv('test_dataset.csv')

# '가-힣0-9'만 남기고 나머지는 제거하는 함수
def clean_text(text):
    cleaned_text = re.sub('[^가-힣0-9]', '', text)
    return cleaned_text.strip()

# attraction 열의 값을 리스트로 변환하고 클린업
places = Tour_Place['attraction'].head(100).tolist()
cleaned_places = [clean_text(place) for place in places]

print(cleaned_places)

['동대문역사문화공원', '서울식물원', '동대문디자인플라자', '남산서울타워', '서울숲', '경복궁', '인사동', '익선동', '명동', '롯데월드어드벤처', '창덕궁과후원유네스코세계유산', '홍대', '덕수궁', '서울선릉과정릉유네스코세계유산', '롯데월드타워서울스카이', '길상사서울', '하늘공원', '동대문종합시장동대문쇼핑타운', '그라운드시소서촌', '종로3가귀금속전문상가', '북촌한옥마을', '낙산공원', '국립중앙박물관', '광화문', '삼각산', '창경궁', '서울한양도성스탬프투어', '스타필드코엑스몰', '송파나루공원석촌호수', '서촌마을', '서울동대문닭한마리골목', '화랑대철도공원', '서울경교장', '문래창작촌', '청계천', '이랜드크루즈한강유람선', '서래마을', '광나루안전체험관', '이태원관광특구', '경희궁', '북촌8경', '손기정문화도서관', '서울로7017', '북악하늘길스카이웨이하늘한마당하늘마루', '돈의문박물관마을', '덕수궁돌담길', '삼성산성지', '한국교회100주년기념관', '하이커그라운드', '망원시장', '레일크루즈해랑열차', '남대문시장', '여의도한강공원', '남산골한옥마을', '경의선숲길연트럴파크', '서울공예박물관', '홍대', '아차산', '서대문형무소역사관', '광장시장', '반포대교달빛무지개분수', '세빛섬', '북악스카이팔각정', '월드컵공원', '올림픽공원', '서울몽촌토성', '서대문자연사박물관', '북서울꿈의숲', '푸른수목원', '송파책박물관', '숭례문', '동대문문구완구거리', '와룡공원', '황학동벼룩시장도깨비시장만물시장', '부암동', '몽마르뜨공원', '충무공이순신동상', '채그로', '대림동차이나타운', '롯데월드타워롯데월드몰', '북정마을', '롯데호텔월드', '봉은사서울', '학전블루소극장', '별마당도서관', '인왕산', '옥천암서울', '한강시민공원뚝섬지구뚝섬한강공원', '종묘유네스코세계유산', '백사실계곡', '불암산', '통인시장', '소소한풍경', '청와대사랑채',

In [7]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time

def get_tourist_info(장소):
    try:
        # 웹 드라이버 실행하고 최대화
        driver = webdriver.Chrome()
        driver.maximize_window()

        # 네이버로 이동 (10초 대기)
        driver.get(f"https://m.search.naver.com/search.naver?sm=mtp_sly.hst&where=m&query={장소}")
        time.sleep(10)

        # 수정된 경로로 검색 버튼 클릭 (10초 대기)
        search_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//*[@id='place-main-section-root']/div/div[2]/div[1]/div[2]/span[2]/a"))  # class="PXMot"
        )

        if '방문자리뷰' in search_button.text:
            time.sleep(5)
            search_button.click()
        else:
            print(f"{장소}: 해당 요소에 '방문자리뷰' 텍스트가 포함되어 있지 않습니다.")
            driver.quit()
            return None

        time.sleep(10)

        # 다음 버튼 클릭 (XPath 사용)
        button_class_name = "dP0sq"

        for i in range(5):  # 최대 5번 클릭 시도
            try:
                first_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CLASS_NAME, button_class_name))
                )
                first_button.click()
                print(f"{i+1}번째 버튼 클릭 성공")
            except TimeoutException:
                print(f"{i+1}번째 버튼이 존재하지 않아 클릭하지 않고 넘어갑니다.")
                break  # 버튼이 더 이상 존재하지 않으면 반복 중단

        # 정보 추출
        results = []
        for j in range(0, 25):  # 0부터 24까지 (25개의 항목)
            try:                   
                good_point = driver.find_elements(By.CLASS_NAME, 't3JSf')[j].text
                select_people = driver.find_elements(By.CLASS_NAME, 'CUoLy')[j].text
                results.append({'good_point': good_point, 'select_people': select_people})
            except IndexError:
                print(f"항목 {j}은(는) 존재하지 않습니다.")

        # 관광지 이름 추출 
        tourist_spot_xpath = '//*[@id="_title"]/div/span[1]'
        tourist_spot = driver.find_element(By.XPATH, tourist_spot_xpath).text

        # 평점 추출
        star_point_xpath = '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[1]/div/div/div[3]/span[1]/em'
        star_point = driver.find_element(By.XPATH, star_point_xpath).text

        # 평점 수 추출        
        total_rating_count_xpath = '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[1]/div/div/div[3]/span[2]'
        total_rating_Count = driver.find_element(By.XPATH, total_rating_count_xpath).text.split()[0]  # '평점수'만 추출하기 위해 split 사용

        # 드라이버 종료
        driver.quit()

        # 데이터 프레임 생성
        df = pd.DataFrame(results)

        # 관광지 이름, 평점, 평점 수 추가
        df['Tourist_Spot'] = tourist_spot
        df['star_point'] = star_point
        df['total_rating_Count'] = total_rating_Count

        print(f"{장소}: 데이터 프레임 생성 성공")

        return df
    
    except Exception as e:
        print(f"{장소}: 오류가 발생했습니다. - {str(e)}")
        if 'driver' in locals():
            driver.quit()
        return None

def process_multiple_places(places):
    combined_df = pd.DataFrame()
    failed_places = []

    for place in places:
        df = get_tourist_info(place)
        if df is not None:
            combined_df = pd.concat([combined_df, df], ignore_index=True)
        else:
            print(f"{place}: 오류가 있습니다.")
            failed_places.append({'place': place, 'error': '오류 발생'})

    fail_df = pd.DataFrame(failed_places)
    print("모든 장소에 대한 데이터 프레임 결합 완료")

    return combined_df, fail_df

# CSV 파일 읽기
Tour_Place = pd.read_csv('test_dataset.csv')

# attraction 열의 값을 리스트로 변환
#places = ['동대문역사문화공원','서울식물원','동대문디자인플라자','인사동문화의거리']
places = ['동대문역사문화공원','서울 식물원','동대문디자인플라자','남산서울타워','서울숲','경복궁','인사동문화의거리',
          '익선동 한옥거리','명동','롯데월드 어드벤처','창덕궁','홍대','선정릉','롯데월드타워 서울스카이',
          '길상사(서울)','서울 하늘공원','동대문 종합시장','그라운드시소 서촌','북촌한옥마을','낙산공원','국립중앙박물관',
          '광화문','삼각산','창경궁','서울한양도성길','스타필드 코엑스몰','송파나루공원','서촌마을','화당대 철도공원','서래마을','서울경교장',
          '문래창작촌', '청계천', '이랜드크루즈한강유람선', '서래마을', '광나루안전체험관', '이태원관광특구', '경희궁', '북촌8경', '손기정문화도서관',
          '서울로7017', '북악하늘길스카이웨이하늘한마당하늘마루', '돈의문박물관마을', '덕수궁돌담길', '삼성산성지', '한국교회100주년기념관', '하이커그라운드',
          '망원시장', '레일크루즈해랑열차', '남대문시장', '여의도한강공원', '남산골한옥마을']
          
#places = Tour_Place['attraction'].head(100).tolist()


# 함수 호출 예시
final_df, fail_df = process_multiple_places(cleaned_places)
final_df.head()

1번째 버튼 클릭 성공
2번째 버튼 클릭 성공
3번째 버튼이 존재하지 않아 클릭하지 않고 넘어갑니다.
항목 14은(는) 존재하지 않습니다.
항목 15은(는) 존재하지 않습니다.
항목 16은(는) 존재하지 않습니다.
항목 17은(는) 존재하지 않습니다.
항목 18은(는) 존재하지 않습니다.
항목 19은(는) 존재하지 않습니다.
항목 20은(는) 존재하지 않습니다.
항목 21은(는) 존재하지 않습니다.
항목 22은(는) 존재하지 않습니다.
항목 23은(는) 존재하지 않습니다.
항목 24은(는) 존재하지 않습니다.
동대문역사문화공원: 데이터 프레임 생성 성공
서울 식물원: 오류가 발생했습니다. - Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF62CD822C2+60002]
	(No symbol) [0x00007FF62CCFCA59]
	(No symbol) [0x00007FF62CBB7EDA]
	(No symbol) [0x00007FF62CC076E6]
	(No symbol) [0x00007FF62CC077AC]
	(No symbol) [0x00007FF62CC4E9D7]
	(No symbol) [0x00007FF62CC2C2CF]
	(No symbol) [0x00007FF62CC4BCC7]
	(No symbol) [0x00007FF62CC2C033]
	(No symbol) [0x00007FF62CBF9657]
	(No symbol) [0x00007FF62CBFA251]
	GetHandleVerifier [0x00007FF62D093E2D+3278285]
	GetHandleVerifier [0x00007FF62D0E0190+3590448]
	GetHandleVerifier [0x00007FF62D0D61D0+3549552]
	GetHandleVerifier [0x00007FF62CE31DE6+779654]
	(No symbol) [0x00007FF62CD07ACF]
	(No symbol) [0x00007FF62CD02EE4

,good_point,select_people,Tourist_Spot,star_point,total_rating_Count
0,"""뷰가 좋아요""",이 키워드를 선택한 인원\n23,동대문역사문화공원,4.27,45개
1,"""관리가 잘 되어있어요""",이 키워드를 선택한 인원\n21,동대문역사문화공원,4.27,45개
2,"""볼거리가 많아요""",이 키워드를 선택한 인원\n14,동대문역사문화공원,4.27,45개
3,"""사진이 잘 나와요""",이 키워드를 선택한 인원\n13,동대문역사문화공원,4.27,45개
4,"""산책로가 잘 되어있어요""",이 키워드를 선택한 인원\n12,동대문역사문화공원,4.27,45개


In [16]:
final_df.to_csv('서촌마을까지.csv')
fail_df.to_csv('서촌마을까지fail.csv')

In [20]:
Tour_Place = pd.read_csv('test_dataset.csv')
places = Tour_Place['attraction'].head(100).tolist()
places

['동대문역사문화공원',
 '서울식물원',
 '동대문디자인플라자(DDP)',
 '남산서울타워',
 '서울숲',
 '경복궁',
 '인사동',
 '익선동',
 '명동',
 '롯데월드 어드벤처',
 '창덕궁과 후원 [유네스코 세계유산]',
 '홍대',
 '덕수궁',
 '서울 선릉과정릉 [유네스코 세계유산]',
 '롯데월드타워 서울스카이',
 '길상사(서울)',
 '하늘공원',
 '동대문 종합시장·동대문 쇼핑타운',
 '그라운드시소 서촌',
 '종로3가 귀금속 전문상가',
 '북촌한옥마을',
 '낙산공원',
 '국립중앙박물관',
 '광화문',
 '삼각산',
 '창경궁',
 '서울 한양도성(스탬프투어)',
 '스타필드 코엑스몰',
 '송파나루공원(석촌호수)',
 '서촌마을',
 '서울 동대문 닭한마리 골목',
 '화랑대 철도공원',
 '서울 경교장',
 '문래창작촌',
 '청계천',
 '이랜드크루즈 (한강유람선)',
 '서래마을',
 '광나루안전체험관',
 '이태원 관광특구',
 '경희궁',
 '북촌 8경',
 '손기정문화도서관',
 '서울로 7017',
 '[북악하늘길 스카이웨이] 하늘한마당~하늘마루',
 '돈의문박물관마을',
 '덕수궁 돌담길',
 '삼성산성지',
 '한국교회 100주년 기념관',
 '하이커 그라운드',
 '망원시장',
 '레일크루즈 해랑열차',
 '남대문시장',
 '여의도한강공원',
 '남산골한옥마을',
 '경의선숲길(연트럴파크)',
 '서울공예박물관',
 'AK&홍대',
 '아차산',
 '서대문형무소역사관',
 '광장시장',
 '반포대교 달빛무지개분수',
 '세빛섬',
 '북악스카이 팔각정',
 '월드컵공원',
 '올림픽공원',
 '서울 몽촌토성',
 '서대문자연사박물관',
 '북서울 꿈의숲',
 '푸른수목원',
 '송파책박물관',
 '숭례문',
 '동대문 문구완구거리',
 '와룡공원',
 '황학동 벼룩시장 (도깨비시장 / 만물시장)',
 '부암동',
 '몽마르뜨공원',
 '충무공 이순신 동상',
 '채그로',
 '대림동 차이나타운',
 '롯데월